In [1]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

# Cinemática Inversa

In [4]:
from kinematicsrobotics.datahandler import Save, Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics

## Base de dados

In [5]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe(r'src\data\ready\dataset-radius-1cm.csv')
dataset

,theta_1,theta_2,theta_3,theta_4,theta_5,p_x,p_y,p_z,roll,pich,yaw
0,0,0,12,0,0,53.213314,-2.477121e-15,2.515179,3.141593e+00,-1.361357,2.752040e-16
1,0,12,12,0,0,53.606657,-1.789652e-15,13.742410,2.378531e+00,-1.570796,7.630613e-01
2,0,12,24,0,0,52.819970,-3.124530e-15,6.257590,3.141593e+00,-1.361357,2.371518e-16
3,0,12,24,12,0,53.213314,-2.915404e-15,10.000000,2.356194e+00,-1.570796,7.853982e-01
4,0,12,36,12,0,51.657132,-4.201082e-15,2.678740,3.141593e+00,-1.361357,2.500234e-16
...,...,...,...,...,...,...,...,...,...,...,...
6884,120,120,120,60,0,-9.000000,1.558846e+01,41.176915,-2.026424e-16,-0.523599,-1.047198e+00
6885,120,120,120,72,0,-7.281153,1.261133e+01,42.707475,-1.899993e-16,-0.314159,-1.047198e+00
6886,120,120,120,84,0,-5.440756,9.423666e+00,43.489851,-1.843716e-16,-0.104720,-1.047198e+00
6887,120,120,120,96,0,-3.559244,6.164791e+00,43.489851,-1.843716e-16,0.104720,-1.047198e+00


In [6]:
size_train,size_val,size_test = 0.7, 0.2, 0.1

In [7]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4']
                        )

In [8]:
x_train, x_test, y_train, y_test = data.data_train_test

## Modelos

In [9]:
mlp = Model.mlp_regressor()
mlp.model


MLPRegressor(early_stopping=True, max_iter=1000, n_iter_no_change=5,
             random_state=42, verbose=True)

## Estimação dos hiperparâmetros

In [19]:
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV, GridSearchCV
from kinematicsrobotics.datahandler import Save
from pandas import DataFrame
from itertools import product

class ParameterOptimizer:
    
    def __init__(self, *, model: Model, x_train, y_train, size_train = 0.7, size_val =  0.1, size_test = 0.2, n_splits: int = 4) -> None:
        self._save = Save()
        self._model = model
        self._x = x_train
        self._y = y_train
        self.size_validation(size_train = size_train, 
                             size_val = size_val,
                             size_test = size_test
        )
        self.holdout(n_splits = n_splits)
    
    # Método público que define a divisão dos dados de treino e validação
    def size_validation(self, *, size_train, size_val, size_test):
        self._size_val = (1 - size_train/(size_train+size_val))
        self._size_train = 1 - self._size_val
        self._size_test = size_test

    # Validação cruzada hold out
    def holdout(self, *, n_splits = 4):
        self._n_splits = n_splits
        
        self._cv = ShuffleSplit(n_splits=n_splits, 
                                test_size = self._size_val, 
                                random_state=42
        )
    
       
class ParameterSearchMLP(ParameterOptimizer):
    def __init__(self, *, min_neurons: int, max_neurons: int, num_layers: int, step: int, **kw) -> None:
        self._min_neurons = min_neurons
        self._max_neurons = max_neurons
        self._num_layers = num_layers
        self._step = step
        super().__init__(**kw)

    def RandomizedSearch(self,*, scoring = 'neg_mean_squared_error', n_iter, path_cv_results = None, path_best_params = None):
        # Configura os parâmetros da técnica de otimização 
        random_search = RandomizedSearchCV(estimator = self._model.model, 
                                           param_distributions = self.param_grid, 
                                           scoring = scoring, 
                                           cv = self._cv, 
                                           n_iter = n_iter, 
                                           random_state = 42, 
                                           return_train_score = True,
                                           verbose = True
        )
        
        # Treina os modelos
        random_search.fit(self._x, self._y)

        # DataFrame que armazena os resultado dos hiperparâmetros
        history = DataFrame(random_search.cv_results_)

        best_params = DataFrame(random_search.best_params_)

        if path_cv_results:
            self._save.dataframe(data = history, path_data = path_cv_results)
        
        if path_best_params:
            self._save.dataframe(data = best_params, path_data = path_best_params)

        # Melhor hiperparâmetro
        
       

        return random_search.best_estimator_
    
    def parameter(self, *, activation):
        param_grid = []
        
        for i, layers in enumerate(self._num_layers):
            hidden_layer = self.space_hidden(layers = layers, 
                                             min_neurons = self._min_neurons[i], 
                                             max_neurons = self._max_neurons[i], 
                                             step = self._step[i]
            )

            param = { 
                'hidden_layer_sizes': hidden_layer,
                'activation': activation
            }
            param_grid.append(param)
 
        self.param_grid = param_grid
            

    def space_hidden(self,*, layers, min_neurons, max_neurons, step):    
        # Gera a lista de possíveis números de neurônios em cada camada
        possible_neurons = list(range(min_neurons, max_neurons + 1, step))

        return list(product(possible_neurons, repeat=layers))

    

In [20]:
# Espaço de busca do grid search
min_neurons= [50, 100, 100, 100]
max_neurons = [3000, 350, 300, 300]
step = [5, 10, 25, 50]
layers = [1, 2, 3, 4]
activation =  ['relu', 'tanh']

cv = ParameterSearchMLP(min_neurons = min_neurons, 
                        max_neurons = max_neurons, 
                        num_layers = layers, 
                        step = step,
                        model = mlp, 
                        x_train = x_train,
                        y_train = y_train,
                        n_splits = 2
)

cv._save._path_project = os.path.abspath(Path().resolve().parent)
cv._save._path_project

cv.parameter(activation = activation)


best_estimator = cv.RandomizedSearch(n_iter = 1, 
                                     path_cv_results = r'src\data\history\parametersearch-MLP\cv_results.csv', 
                                     path_best_params = r'src\data\history\parametersearch-MLP\best_params.csv'
                                     
)



Fitting 2 folds for each of 1 candidates, totalling 2 fits
Iteration 1, loss = 0.28446713
Validation score: 0.506161
Iteration 2, loss = 0.23685149
Validation score: 0.525045
Iteration 3, loss = 0.23352918
Validation score: 0.530769
Iteration 4, loss = 0.23191838
Validation score: 0.523460
Iteration 5, loss = 0.23219462
Validation score: 0.525832
Iteration 6, loss = 0.23335716
Validation score: 0.527574
Iteration 7, loss = 0.23248570
Validation score: 0.528995
Iteration 8, loss = 0.23219028
Validation score: 0.528386
Iteration 9, loss = 0.23222707
Validation score: 0.529000
Validation score did not improve more than tol=0.000100 for 5 consecutive epochs. Stopping.
Iteration 1, loss = 0.28302664
Validation score: 0.521843
Iteration 2, loss = 0.23685831
Validation score: 0.534054
Iteration 3, loss = 0.23409416
Validation score: 0.533197
Iteration 4, loss = 0.23380688
Validation score: 0.538896
Iteration 5, loss = 0.23408157
Validation score: 0.534215
Iteration 6, loss = 0.23402959
Valida

In [21]:
best_params

{'hidden_layer_sizes': (300, 200, 300, 300), 'activation': 'tanh'}

In [70]:
print(f"1 chamada:{len(cv.param_grid[0]['hidden_layer_sizes'])}\n2 chamada: {len(cv.param_grid[1]['hidden_layer_sizes'])}\n3 chamada:{len(cv.param_grid[2]['hidden_layer_sizes'])}\n4 chamada:{len(cv.param_grid[3]['hidden_layer_sizes'])}")

1 chamada:591
2 chamada: 676
3 chamada:729
4 chamada:625


In [68]:
best_estimator

MLPRegressor(early_stopping=True, hidden_layer_sizes=(200, 250, 275),
             max_iter=1000, n_iter_no_change=5, random_state=42, verbose=True)

cv_results_
best_estimator_
best_params_

In [ ]:
from sklearn.model_selection import ShuffleSplit,RandomizedSearchCV,GridSearchCV
from itertools import product
from sklearn.metrics import mean_squared_error,r2_score
from pandas import DataFrame

# Calcula a porcentagem relativa dos dados de treino e validação
def size_split(size_train,size_val):
    size_val = (1 - size_train/(size_train+size_val))
    return size_val

# Espaço de busca dos neurônios
def space_hidden(min_neurons, max_neurons, layers, step):
    # Lista que armazena os neurônios
    neuron_space = []
    neuron_combinations = []
    
    # Gera a lista de possíveis números de neurônios em cada camada
    possible_neurons = list(range(min_neurons, max_neurons + 1,step))
    
    # Crie todas as combinações possíveis
    for num_layers in layers:
        neuron_combinations(list(product(possible_neurons, repeat=num_layers)))
        
    neuron_space.extend(neuron_combinations)
    return neuron_space

# Parâmetro da rede mlp
def parameter_mlp(min_neurons,max_neurons,layers,step,func_act): 
    # Dicionário que armazena os hiperparâmetros
    param_grid = {
        'hidden_layer_sizes': space_hidden(min_neurons, max_neurons,layers,step),
        'activation': func_act
    }

    return param_grid

# Random Search
def RandomizedSearch(x, y, model, param_grid, scoring,cv, n_iter):
    # Configura os parâmetros da técnica de otimização 
    random_search = RandomizedSearchCV(estimator=model, 
                                       param_distributions=param_grid, 
                                       scoring=scoring, 
                                       cv=cv, 
                                       n_iter=n_iter, 
                                       random_state=42, 
                                       return_train_score=True,
                                       verbose=0)
    
    # Treina os modelos
    random_search.fit(x, y)

    # DataFrame que armazena os resultado dos hiperparâmetros
    history = DataFrame(random_search.cv_results_)

    # Melhor hiperparâmetro
    best_estimator = random_search.best_estimator_

    return history,best_estimator

# Grid Search
def GridSearch(x, y, model, param_grid, scoring, cv):
    # Configura os parâmetros da técnica de otimização 
    gridshearch = GridSearchCV(estimator=model,
                               param_grid=param_grid,
                               cv=cv,
                               scoring=scoring,
                               verbose=0,
                               return_train_score=True)
    
    # Treina os modelos
    gridshearch.fit(x, y)
    
    # Armazena os resultado dos hiperparâmetros
    history = DataFrame(gridshearch.cv_results_)
    
    best_estimator = gridshearch.best_estimator_

    return history,best_estimator

# Validação cruzada hold out
def holdout(n_splits,test_size):
    return ShuffleSplit(n_splits=n_splits, 
                      test_size=test_size, 
                      random_state=42)



In [ ]:
def space_hidden(min_neurons, max_neurons, layers, step):    
    # Gera a lista de possíveis números de neurônios em cada camada
    possible_neurons = list(range(min_neurons, max_neurons + 1,step))

    return list(product(possible_neurons, repeat=layers))

# Parâmetro da rede mlp
def parameter_mlp(min_neurons,max_neurons,layers,step,func_act): 
    # Dicionário que armazena os hiperparâmetros
    param_grid = {
        'hidden_layer_sizes': space_hidden(min_neurons, max_neurons,layers,step),
        'activation': func_act
    }

    return param_grid

# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = 2
func_act =  ['relu', 'tanh']
parameter_mlp(min_neurons,max_neurons,layers,step,func_act)

In [ ]:
# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = [2,3]
func_act =  ['relu', 'tanh']

param_grid = parameter_mlp(min_neurons = min_neurons,
                           max_neurons = max_neurons,
                           layers=layers,
                           step=step,
                           func_act=func_act)

len(param_grid['hidden_layer_sizes'])

In [ ]:
# Salvar as métricas 
sv = save(path_project[0])
path_data_save =  r'src\data\ready\history.csv'

# Dados de treino e validação
size_val = size_split(size_train,size_val)

# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = [2,3]
func_act =  ['relu', 'tanh']

param_grid = parameter_mlp(min_neurons = min_neurons,
                           max_neurons = max_neurons,
                           layers=layers,
                           step=step,
                           func_act=func_act)


# validação cruzada hold out
n_splits = 5

# Número de amostras selecionadas no espaço de busca
n_iter = 1000

# Técnica de validação
cv = holdout(n_splits,size_val)

# inicializando a rede
mlp = MLP()

# APlicação do grid search
history,best_model = RandomizedSearch(x=x_train, 
                           y=y_train, 
                           model=mlp, 
                           param_grid=param_grid,
                           scoring = 'neg_mean_squared_error', 
                           cv=cv,
                           n_iter = n_iter)


sv.dataframe(history,path_data_save)



### Infomações

In [ ]:
print(f"Número de parametros de camada oculta: {len(param_grid['hidden_layer_sizes'])}")
print(f"Atributos do retorno da gridsearsh: {history.columns}")

In [ ]:
best_model

In [ ]:
history[['mean_test_score','std_test_score','mean_train_score', 'std_train_score','params']]